<a href="https://colab.research.google.com/github/mehdiz5/instadeep-haick-challenge/blob/main/instadeep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip train.csv.zip

Archive:  train.csv.zip
  inflating: train.csv               


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 70.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 92.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from torch.utils.data import DataLoader, Dataset

In [ ]:
data = pd.read_csv("train.csv")

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
data.head()

,ID,protein_name,sequence,biological_process,cellular_component,molecular_function
0,train_0,sp|Q9J523|V212_FOWPN,MPQNKVLSFPLPEGTLLEDITKNKWILGKQLGSGGFGLVYQVSCKS...,0,0,1
1,train_1,sp|Q14990|ODFP1_HUMAN,MAALSCLLDSVRRDIKKVDRELRQLRCIDEFSTRCLCDLYMHPYCC...,0,0,1
2,train_2,sp|Q9C994|DTX14_ARATH,MDSAEKGLLVVSDREEVNKKDGFLRETKKLSYIAGPMIAVNSSMYV...,0,0,1
3,train_3,sp|Q58425|CARA_METJA,MEAVLILEDGTILKGKGFGAEKEVFGELVFTTVMTGYVEVLTDPSY...,0,1,0
4,train_4,sp|P78814|SRP2_SCHPO,MSETRLFVGRIPPQATREDMMDFFKGYGQILDCKLMNGFGFVEVED...,0,0,1


In [ ]:
#data["sequence"] = data["sequence"].apply(lambda x: " ".join(list(x)))

In [ ]:
data["label"] = data.iloc[:, 3:].apply(lambda x: ''.join(x.astype(str)), axis=1)

In [ ]:
data["label"] = data.label.map({'001':0 ,'010': 1, '100': 2})

In [ ]:
sequences = data.sequence.values
labels = data.label.values

In [ ]:
from sklearn.model_selection import train_test_split
train_sequences, test_sequences, train_labels, test_labels = train_test_split(sequences, labels, test_size=0.2, random_state=42)

In [ ]:
class ProteinSequenceDataset(Dataset):
    def __init__(self, sequences, labels, tokenizer):
        self.sequences = sequences
        self.labels = labels
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        sequence = self.sequences[idx]
        label = self.labels[idx]

        return {'sequence': sequence  'labels': torch.tensor(label)}

In [ ]:
import torch.nn as nn
from transformers import AutoTokenizer, AutoModel

class ProtbertLSTM(nn.Module):
    def __init__(self, num_classes=3, hidden_dim=128):
        super().__init__()
        self.tokenizer = AutoTokenizer.from_pretrained("Rostlab/prot_bert_bfd")
        self.encoder = AutoModel.from_pretrained("Rostlab/prot_bert_bfd")
        self.linear = nn.Linear(768, 128)
        self.lstm = nn.LSTM(input_size=128, hidden_size=hidden_dim, num_layers=1, batch_first=True)
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, sequences):
        # Encode protein sequences using ProtBERT
        encoded = self.tokenizer(sequences, padding=True, truncation=True, return_tensors='pt')
        input_ids = encoded['input_ids']
        attention_mask = encoded['attention_mask']

        # Get embeddings from the ProtBERT encoder
        with torch.no_grad():
            model_outputs = self.encoder(input_ids, attention_mask=attention_mask)
        embeddings = self.linear(model_outputs[0])

        # Feed embeddings into LSTM
        lstm_output, _ = self.lstm(embeddings)

        # Classify using fully connected layer
        final_output = self.fc(lstm_output[:, -1, :])

        return final_output

In [ ]:
model = ProtbertLSTM()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
train_dataset = ProteinSequenceDataset(train_sequences, train_labels, tokenizer)
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(test_sequences, batch_size=32, shuffle=False)

NameError: ignored

In [ ]:
def train_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    train_loss = 0.0
    num_correct = 0
    num_total = 0

    for batch in dataloader:
        print(batch)
        sequences, labels = batch
        sequences = sequences.to(device)
        labels = labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(sequences)
        loss = criterion(outputs, labels)
        train_loss += loss.item()

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Calculate accuracy
        _, predictions = torch.max(outputs, dim=1)
        num_correct += torch.sum(predictions == labels)
        num_total += len(labels)

    # Calculate metrics
    avg_loss = train_loss / len(dataloader)
    avg_acc = num_correct / num_total
    return avg_loss, avg_acc

In [ ]:
def evaluate(model, dataloader, criterion, device):
    model.eval()
    val_loss = 0.0
    num_correct = 0
    num_total = 0

    with torch.no_grad():
        for batch in dataloader:
            print(batch)
            sequences, labels = batch
            sequences = sequences.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(sequences)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            # Calculate accuracy
            _, predictions = torch.max(outputs, dim=1)
            num_correct += torch.sum(predictions == labels)
            num_total += len(labels)

    # Calculate metrics
    avg_loss = val_loss / len(dataloader)
    avg_acc = num_correct / num_total
    return avg_loss, avg_acc

In [ ]:
num_epochs = 10
for epoch in range(num_epochs):
    # Train for one epoch
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
    
    # Evaluate on validation set
    val_loss, val_acc = evaluate(model, val_loader, criterion, device)
    
    # Print training and validation metrics
    print(f"Epoch {epoch+1}/{num_epochs}:")
    print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.4f}")
    print(f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_acc:.4f}")

['MAIWFARSKTLVSSLRHNLNLSTILIKRDYSHRPIFYTTSQLSSTAYLSPFGSLRHESTAVETQADHLVQQIDEVDAQELDFPGGKVGYTSEMKFIPESSSRRIPCYRVLDEDGRIIPDSDFIPVSEKLAVRMYEQMATLQVMDHIFYEAQRQGRISFYLTSVGEEAINIASAAALSPDDVVLPQYREPGVLLWRGFTLEEFANQCFGNKADYGKGRQMPIHYGSNRLNYFTISSPIATQLPQAAGVGYSLKMDKKNACTVTFIGDGGTSEGDFHAGLNFAAVMEAPVVFICRNNGWAISTHISEQFRSDGIVVKGQAYGIRSIRVDGNDALAVYSAVRSAREMAVTEQRPVLIEMMTYRVGHHSTSDDSTKYRAADEIQYWKMSRNPVNRFRKWVEDNGWWSEEDESKLRSNARKQLLQAIQAAEKWEKQPLTELFNDVYDVKPKNLEEQELGLKELVKKQPQDYPPGFHV', 'MSSAYCSSAVAVSAAATASSAATFNPLLSSHSNSQLFYRFTPKSFKLVANCPNPLILHSNIRRHRFFCAAETEASSADDEIQASVEEEEEVEEEGDEGEEEVEEEKQTTQASGEEGRLYVGNLPYTITSSELSQIFGEAGTVVDVQIVYDKVTDRSRGFGFVTMGSIEEAKEAMQMFNSSQIGGRTVKVNFPEVPRGGENEVMRTKIRDNNRSYVDSPHKVYAGNLGWNLTSQGLKDAFGDQPGVLGAKVIYERNTGRSRGFGFISFESAENVQSALATMNGVEVEGRALRLNLASEREKPTVSPPSVEEGETEEASLESNEVLSNVSA', 'MCCYLNLLMIQFESSHICHWFWNTPYRALQRAYKASKKVRNIHTNYIFCKSKPAFQRFGYNLDLYIDSILDESSFQIYWGLLEFKASRFVLTKFSNLIFKHNFHLLTQTDGNKLFSNSFDREQSLLFCDIHSTSLKIINRKLAWIEAALADLEMLRDNSSSTSITPILNKVYNVSLPMSLDSTSKRVAY

ValueError: ignored